In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import style
#style.use('fivethirtyeight')
import seaborn as sns
#sns.set(style = 'white', color_code=True)
# import warnings
# warnings.filterwarnings('ignore')
# sns.set_style('whitegrid')

from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Performing grid search
# from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [2]:
df = pd.read_csv('train.csv')
df['Gender'] = df['Gender'].fillna( df['Gender'].dropna().mode().values[0] )
df['Married'] = df['Married'].fillna( df['Married'].dropna().mode().values[0] )
df['Dependents'] = df['Dependents'].fillna( df['Dependents'].dropna().mode().values[0] )
df['Self_Employed'] = df['Self_Employed'].fillna( df['Self_Employed'].dropna().mode().values[0] )
df['LoanAmount'] = df['LoanAmount'].fillna( df['LoanAmount'].dropna().mean() )
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna( df['Loan_Amount_Term'].dropna().mode().values[0] )
df['Credit_History'] = df['Credit_History'].fillna( df['Credit_History'].dropna().mode().values[0] )
df['Dependents'] = df['Dependents'].str.rstrip('+')
df['Gender'] = df['Gender'].map({'Female':0,'Male':1}).astype(np.int)
df['Married'] = df['Married'].map({'No':0, 'Yes':1}).astype(np.int)
df['Education'] = df['Education'].map({'Not Graduate':0, 'Graduate':1}).astype(np.int)
df['Self_Employed'] = df['Self_Employed'].map({'No':0, 'Yes':1}).astype(np.int)
#df['Loan_Status'] = df['Loan_Status'].map({'N':0, 'Y':1}).astype(np.int)
df['Dependents'] = df['Dependents'].astype(np.int)
#Property Map
property_map = {'Semiurban':77,'Urban':66,'Rural':62}
df['Property_Area'] = df['Property_Area'].map(property_map)
# test_loan_df['Property_Area']  = test_loan_df['Property_Area'].map(property_map)
#df.drop(['Dependents','Gender','Propert_Area','Married','Education','Self_Employed'],axis=1,inplace=True)

In [4]:
dtest = pd.read_csv('test.csv')

dtest['Gender'] = dtest['Gender'].map({'Female':0,'Male':1})
dtest['Married'] = dtest['Married'].map({'No':0, 'Yes':1}).astype(np.int)
dtest['Education'] = dtest['Education'].map({'Not Graduate':0, 'Graduate':1}).astype(np.int)
dtest['Self_Employed'] = dtest['Self_Employed'].map({'No':0, 'Yes':1})
dtest['Dependents'] = dtest['Dependents'].str.rstrip('+')
dtest['Gender'] = dtest['Gender'].fillna( dtest['Gender'].dropna().mode().values[0]).astype(np.int)
dtest['Dependents'] = dtest['Dependents'].fillna( dtest['Dependents'].dropna().mode().values[0]).astype(np.int)
dtest['Self_Employed'] = dtest['Self_Employed'].fillna( dtest['Self_Employed'].dropna().mode().values[0])
dtest['LoanAmount'] = dtest['LoanAmount'].fillna( dtest['LoanAmount'].dropna().mode().values[0])
dtest['Loan_Amount_Term'] = dtest['Loan_Amount_Term'].fillna( dtest['Loan_Amount_Term'].dropna().mode().values[0])
dtest['Credit_History'] = dtest['Credit_History'].fillna( dtest['Credit_History'].dropna().mode().values[0] )
#Property Map
property_map = {'Semiurban':77,'Urban':66,'Rural':62}
dtest['Property_Area']  = dtest['Property_Area'].map(property_map)
#dtest.drop(['Dependents','Gender','Propert_Area','Married','Education','Self_Employed'],axis=1,inplace=True)

In [6]:
dtest.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,1,1,0,1,0.0,5720,0,110.0,360.0,1.0,66
1,LP001022,1,1,1,1,0.0,3076,1500,126.0,360.0,1.0,66
2,LP001031,1,1,2,1,0.0,5000,1800,208.0,360.0,1.0,66
3,LP001035,1,1,2,1,0.0,2340,2546,100.0,360.0,1.0,66
4,LP001051,1,0,0,0,0.0,3276,0,78.0,360.0,1.0,66


In [5]:
X_test = dtest.iloc[:,1:]
X_test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1,1,0,1,0.0,5720,0,110.0,360.0,1.0,66
1,1,1,1,1,0.0,3076,1500,126.0,360.0,1.0,66
2,1,1,2,1,0.0,5000,1800,208.0,360.0,1.0,66
3,1,1,2,1,0.0,2340,2546,100.0,360.0,1.0,66
4,1,0,0,0,0.0,3276,0,78.0,360.0,1.0,66


In [ ]:
target = 'Loan_Status'
IDcol = 'Loan_ID'

In [ ]:
clf_final = GradientBoostingClassifier(learning_rate =0.001,random_state=10,subsample=0.7,n_estimators=2000,max_features='sqrt',
                                 max_depth=5,min_samples_leaf=13,min_samples_split=50)
predictors = [x for x in df.columns if x not in [target, IDcol]]

# modelfit(clf, df, predictors)
clf_final.fit(df[predictors], df['Loan_Status'])
dtrain_predictions = clf_final.predict(X_test)

In [ ]:
submission = pd.DataFrame({
        'Loan_ID':dtest['Loan_ID'],
        'Loan_Status':dtrain_predictions
    })
submission.to_csv('Loan_Prediction.csv',index=False)